# Markov Decision Processes (MDP)
## Future Return

In [1]:
import numpy as np
import random
import gym
import tqdm
import torch
import torch.nn as nn

# for reproductability
torch.manual_seed(0)
random.seed(0)

In [2]:
def calculate_naive_returns(rewards):
    """ Calculates a list of naive returns given a 
    list of rewards."""
    total_returns = np.zeros(len(rewards))
    total_return = 0.0
    for t in range(len(rewards), 0):
        total_return = total_return + reward
        total_returns[t] = total_return
    return total_returns


def discount_rewards(rewards, gamma=0.98):
    discounted_returns = [0 for _ in rewards]
    discounted_returns[-1] = rewards[-1]
    # iterate backwards
    for t in range(len(rewards)-2, -1, -1): 
        discounted_returns[t] = (rewards[t] + 
              discounted_returns[t+1]*gamma)
    return discounted_returns

def epsilon_greedy_action(action_distribution, 
                          epsilon=1e-1):
    action_distribution = action_distribution.detach().numpy()
    if random.random() < epsilon:
        return np.argmax(np.random.random(
           action_distribution.shape))
    else:
        return np.argmax(action_distribution)

def epsilon_greedy_action_annealed(action_distribution,
                                   percentage, 
                                   epsilon_start=1.0, 
                                   epsilon_end=1e-2):
    action_distribution = action_distribution.detach().numpy()
    annealed_epsilon = (epsilon_start*(1.0-percentage) + 
                        epsilon_end*percentage)
    if random.random() < annealed_epsilon:
        return np.argmax(np.random.random(
          action_distribution.shape))
    else:
        return np.argmax(action_distribution)

# Pole-Cart with Policy Gradients

## Creating an Agent

In [3]:
from torch import optim
class PGAgent(object):
    def __init__(self, state_size, num_actions, 
                 hidden_size, 
                 learning_rate=1e-3, 
                 explore_exploit_setting= \
                 'epsilon_greedy_annealed_1.0->0.001'):
        self.state_size = state_size
        self.num_actions = num_actions
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        self.explore_exploit_setting = \
                        explore_exploit_setting
        self.build_model()


    def build_model(self):
      self.model = torch.nn.Sequential(
        nn.Linear(self.state_size, self.hidden_size),
        nn.Linear(self.hidden_size, self.hidden_size),
        nn.Linear(self.hidden_size, self.num_actions),
        nn.Softmax(dim=0))

    def train(self, state, action_input, reward_input):
        state = torch.tensor(state).float()
        action_input = torch.tensor(action_input).long()
        reward_input = torch.tensor(reward_input).float()
        self.output = self.model(state)
        # Select the logits related to the action taken
        logits_for_actions = self.output.gather(1,action_input.view(-1,1))

        self.loss = -torch.mean(
            torch.log(logits_for_actions) * reward_input)
        self.loss.backward()
        self.optimizer = optim.Adam(self.model.parameters())
        self.optimizer.step()
        self.optimizer.zero_grad()
        return self.loss.item()
        


    def sample_action_from_distribution(self, 
                                        action_distribution, 
                                        epsilon_percentage):
        # Choose an action based on the action probability
        # distribution and an explore vs exploit
        if self.explore_exploit_setting == 'greedy':
              action = epsilon_greedy_action(action_distribution,0.00)
        elif self.explore_exploit_setting == 'epsilon_greedy_0.05':
              action = epsilon_greedy_action(action_distribution,0.05)
        elif self.explore_exploit_setting == 'epsilon_greedy_0.25':
              action = epsilon_greedy_action(action_distribution,0.25)
        elif self.explore_exploit_setting == 'epsilon_greedy_0.50':
              action = epsilon_greedy_action(action_distribution,0.50)
        elif self.explore_exploit_setting == 'epsilon_greedy_0.90':
              action = epsilon_greedy_action(action_distribution,0.90)
        elif self.explore_exploit_setting == \
          'epsilon_greedy_annealed_1.0->0.001':
              action = epsilon_greedy_action_annealed(
                  action_distribution, 
                  epsilon_percentage, 1.0,0.001)
        elif self.explore_exploit_setting == \
          'epsilon_greedy_annealed_0.5->0.001':
              action = epsilon_greedy_action_annealed(
                  action_distribution, 
                  epsilon_percentage, 0.5, 0.001)
        elif self.explore_exploit_setting == \
          'epsilon_greedy_annealed_0.25->0.001':
              action = epsilon_greedy_action_annealed(
                  action_distribution, 
                  epsilon_percentage, 0.25, 0.001)
        return action

    def predict_action(self, state, epsilon_percentage):
        action_distribution = self.model(torch.from_numpy(state).float())
        action = self.sample_action_from_distribution(
            action_distribution, epsilon_percentage)
        return action

## Keeping Track of History

In [4]:
class EpisodeHistory(object):

    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.state_primes = []
        self.discounted_returns = []

    def add_to_history(self, state, action, reward, 
      state_prime):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.state_primes.append(state_prime)


class Memory(object):

    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.state_primes = []
        self.discounted_returns = []

    def reset_memory(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.state_primes = []
        self.discounted_returns = []

    def add_episode(self, episode):
        self.states += episode.states
        self.actions += episode.actions
        self.rewards += episode.rewards
        self.discounted_returns += episode.discounted_returns

## Policy Gradient Main Function

In [5]:
# Configure Settings
#total_episodes = 5000
total_episodes = 16
total_steps_max = 10000
epsilon_stop = 3000
train_frequency = 8
max_episode_length = 500
render_start = -1
should_render = False

explore_exploit_setting = 'epsilon_greedy_annealed_1.0->0.001'

env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]  # 4 for 
                                              # CartPole-v0
num_actions = env.action_space.n  # 2 for CartPole-v0

solved = False
agent = PGAgent(state_size=state_size,
                num_actions=num_actions,
                hidden_size=16, 
                explore_exploit_setting= \
                  explore_exploit_setting)

In [6]:
episode_rewards = []
batch_losses = []

global_memory = Memory()
steps = 0
for i in range(total_episodes):
  state = env.reset()
  episode_reward = 0.0
  episode_history = EpisodeHistory()
  epsilon_percentage = float(min(i/float(epsilon_stop), 1.0))

  for j in range(max_episode_length):
      action = agent.predict_action(state, epsilon_percentage)
      state_prime, reward, terminal, _ = env.step(action)
      
      episode_history.add_to_history(
          state, action, reward, state_prime)
      state = state_prime
      episode_reward += reward
      steps += 1
      
      if j == (max_episode_length - 1):
            terminal = True
            
      if terminal:
          episode_history.discounted_returns = \
            discount_rewards(episode_history.rewards)
          global_memory.add_episode(episode_history)

          # every 8th episode train the NN
          # train on all actions from episodes in memory, then reset memory
          if np.mod(i, train_frequency) == 0:
            reward_input = global_memory.discounted_returns
            action_input = global_memory.actions
            state = global_memory.states

            # train step 
            batch_loss = agent.train(state, action_input, reward_input)
              # print(f'Batch loss: {batch_loss}')
              # batch_losses.append(batch_loss)
            global_memory.reset_memory()

          episode_rewards.append(episode_reward)

          if i % 10 == 0:
              mean_rewards = torch.mean(torch.tensor(episode_rewards[:-10]))
              if mean_rewards > 10.0:
                  solved = True
              else:
                  solved = False
              print(f'Solved: {solved} Mean Reward: {mean_rewards}')
          break # stop playing if terminal
          
  print(f'Episode[{i}]: {len(episode_history.actions)} \
          actions {episode_reward} reward')

Solved: False Mean Reward: nan
Episode[0]: 11           actions 11.0 reward
Episode[1]: 14           actions 14.0 reward
Episode[2]: 14           actions 14.0 reward
Episode[3]: 18           actions 18.0 reward
Episode[4]: 22           actions 22.0 reward
Episode[5]: 15           actions 15.0 reward
Episode[6]: 17           actions 17.0 reward
Episode[7]: 12           actions 12.0 reward
Episode[8]: 38           actions 38.0 reward
Episode[9]: 14           actions 14.0 reward
Solved: True Mean Reward: 11.0
Episode[10]: 15           actions 15.0 reward
Episode[11]: 19           actions 19.0 reward
Episode[12]: 16           actions 16.0 reward
Episode[13]: 18           actions 18.0 reward
Episode[14]: 19           actions 19.0 reward
Episode[15]: 27           actions 27.0 reward


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
